In [ ]:
# default_exp core

# XG Boost

> Task descriptions for XG Boost test project.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def load_dataframe() -> "ray.ObjectRef":
    """
    #### build random dataframe task

    """
    print("Loading CSV.")
    if SIMPLE:
        print("Loading simple")
        from sklearn import datasets
        data = datasets.load_breast_cancer(return_X_y=True)
    else:
        import pandas as pd
        # import modin.pandas as mpd -- currently does not work.
        url = "https://archive.ics.uci.edu/ml/machine-learning-databases/" \
         "00280/HIGGS.csv.gz"
        colnames = ["label"] + ["feature-%02d" % i for i in range(1, 29)]
        data = pd.read_csv(url, compression='gzip', names=colnames)
        # data = pd.read_csv("/home/astro/HIGGS.csv.gz", names=colnames)
        print("loaded higgs")
    print("Loaded CSV.")
    return data

In [ ]:
#export
def create_data(data):
    print("RUNNING SOME CODE!")
    logfile = open("/tmp/ray/session_latest/custom.log", "w")
    def write(msg):
        logfile.write(f"{msg}\n")
        logfile.flush()

    write(f"Creating data matrix: {data, SIMPLE}")
    if SIMPLE:
        from sklearn.model_selection import train_test_split
        write("Splitting data")
        data, labels = data
        train_x, test_x, train_y, test_y = train_test_split(
            data, labels, test_size=0.25)

        train_set = xgb.RayDMatrix(train_x, train_y)
        test_set = xgb.RayDMatrix(test_x, test_y)
    else:
        df_train = data[(data['feature-01'] < 0.4)]
        colnames = ["label"] + ["feature-%02d" % i for i in range(1, 29)]
        train_set = xgb.RayDMatrix(df_train, label="label", columns=colnames)
        df_validation = data[(data['feature-01'] >= 0.4)& (data['feature-01'] < 0.8)]
        test_set = xgb.RayDMatrix(df_validation, label="label")
    write("finished data matrix")
    return train_set, test_set

In [ ]:
#export
def train_model(data) -> None:
        logfile = open("/tmp/ray/session_latest/custom.log", "w")
        def write(msg):
            logfile.write(f"{msg}\n")
            logfile.flush()

        dtrain, dvalidation = data
        evallist = [(dvalidation, 'eval')]
        evals_result = {}
        config = {
            "tree_method": "hist",
            "eval_metric": ["logloss", "error"],
        }
        write("Start training")
        bst = xgb.train(
            params=config,
            dtrain=dtrain,
            evals_result=evals_result,
            ray_params=xgb.RayParams(max_actor_restarts=1, num_actors=2, cpus_per_actor=2),
            num_boost_round=100,
            evals=evallist)
        write("finish training")
        return bst
